In [3]:
import os, warnings
import wandb
import pandas as pd
from fastai.vision.all import *
from sklearn.model_selection import StratifiedGroupKFold
import params
warnings.filterwarnings('ignore')


Data preparation

En este notebook vamos a perepara los datos para el entrenamiento de los modelos. Para ello, vamos a realizar las siguientes tareas:
* vamos a descargar la data artifical generada en el notebook anterior con wandb
* vamos a dividir la data en train y test
* vamos a guardar el split en la tabla de EDA que habíamos creado anteriormente

In [4]:
run = wandb.init(project=params.WANDB_PROJECT, entity=params.ENTITY, job_type="data_split")

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: dramisauria. Use `wandb login --relogin` to force relogin


In [5]:
raw_data_at = run.use_artifact(f'{params.RAW_DATA_AT}:latest')
path = Path(raw_data_at.download())

wandb: Downloading large artifact bdd_simple_1k:latest, 846.57MB. 4007 files... 
wandb:   4007 of 4007 files downloaded.  
Done. 0:0:1.8


In [6]:
path #todo melo

Path('artifacts/bdd_simple_1k:v0')

In [7]:
path.ls() #artifacts disponibles

(#5) [Path('artifacts/bdd_simple_1k:v0/images'),Path('artifacts/bdd_simple_1k:v0/labels'),Path('artifacts/bdd_simple_1k:v0/LICENSE.txt'),Path('artifacts/bdd_simple_1k:v0/eda_table.table.json'),Path('artifacts/bdd_simple_1k:v0/media')]

Para dividir la data en train, test y val, necesitamos obviamente los nombres de los files, para eso vamos a usar las columnas que teníamos al momento de crear la eda table. 

In [8]:
fnames = os.listdir(path/'images')
groups = [s.split('-')[0] for s in fnames]

In [15]:
orig_eda_table = raw_data_at.get("eda_table")

wandb: Downloading large artifact bdd_simple_1k:latest, 846.57MB. 4007 files... 
wandb:   4007 of 4007 files downloaded.  
Done. 0:0:0.8


In [16]:
orig_eda_table

In [17]:
y = orig_eda_table.get_column('bicycle')


A continuación vamos a dividir la data en train 80%, validación 10% y test de 10%.
Hay que tratar de evitar fugas, vamos a ir agrupando datos según el identificador de video (queremos asegurarnos de que nuestro modelo pueda generalizarse a autos nuevos o fotogramas de video). Por lo anterior, es importante manejar el desequilibrio de etiquetas y por ende, estratificamos los datos con nuestra columna objetivo. Para mayor información, te lo explico en el readme para que entiendas a qué hace referencia esto y porque es importante, no le veo mucho chiste manejar tools de mlops, si no sabes cómo dianosticar un modelo de ML, generar estrategias para solucionarlo y evaluar si funcionan. 

Vamos a usar StratidiedGroupKFold para dividir los datos en 10, 8 para train, uno para test y otro para validación. los folds hace referencia a la cantidad de grupos en los que vamos a dividir los datos, en este caso 10. Lo anterior, garantiza que los grupos se mantengan intactos en cada pliegue para evitar que se dividan entre el conjunto de entrenamiento y prueba. Es decir, si las imágenes que provienen de la misma cámara (o grupo o fuente o id o whatever) se mantengan juntas en la misma partición (pliegue) durante la validación cruzada. Esto significa que, en cada pliegue, todas las imágenes tomadas por una cámara específica estarán en el mismo conjunto de entrenamiento o en el mismo conjunto de prueba, pero no se dividirán entre los dos conjuntos.

In [21]:
df = pd.DataFrame()
df["File_Name"] = fnames
df["fold"] = -1

In [22]:
df.head(2)

,File_Name,fold
0,a59131a5-00000000.jpg,-1
1,6886b3d9-6ab2b28d.jpg,-1


In [26]:
fnames[:3]

['a59131a5-00000000.jpg', '6886b3d9-6ab2b28d.jpg', '115e4aff-00000000.jpg']

In [25]:
groups[0:5]

['a59131a5', '6886b3d9', '115e4aff', 'b803d91d', 'c665137e']

In [27]:
CV = StratifiedGroupKFold(n_splits=10)
for i, (train_idxs, test_idxs) in enumerate(CV.split(df, y, groups)):
    df.loc[test_idxs, "fold"] = i


In [34]:
df['Stage'] = 'train' # 8 folds
df.loc[df.fold == 0, ['Stage']] = 'test' #1 fold
df.loc[df.fold == 1, ['Stage']] = 'valid' # 1 fold
del df['fold']

In [35]:
df.to_csv('data_split.csv', index=False)

In [36]:
path

Path('artifacts/bdd_simple_1k:v0')

In [37]:
# Ahora qué? pues subir el artifact con la data split :) 
processed_data_at = wandb.Artifact(name = params.PROCESSED_DATA_AT, type="split_data")
processed_data_at.add_file('data_split.csv')
processed_data_at.add_dir(path)


wandb: Adding directory to artifact (./artifacts/bdd_simple_1k:v0)... Done. 2.0s


Finalmente, la información dividida puede ser relevante para nuestros análisis: en lugar de cargar imágenes nuevamente, guardaremos la información dividida en una nueva tabla y la uniremos a la tabla EDA que creamos anteriormente.

In [38]:
data_split_table = wandb.Table(dataframe=df[['File_Name', 'Stage']])

In [39]:
join_table = wandb.JoinedTable(orig_eda_table, data_split_table, "File_Name")


In [40]:
processed_data_at.add(join_table, "eda_table_data_split")


ArtifactManifestEntry(path='eda_table_data_split.joined-table.json', digest='OO3tyCvaFFzppiIKTRuBCw==', ref=None, birth_artifact_id=None, size=123, extra={}, local_path='/Users/mdurango/Library/Application Support/wandb/artifacts/staging/tmplazlyx10')

In [41]:
run.log_artifact(processed_data_at) #preparamos carga
run.finish()

Ya me imagino una unet o un sam model, pero antes vamos a ir por un baseline. 
